In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")

In [4]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [5]:
transactions_train.shape

(31788324, 8)

In [106]:
filtered_df = transactions_train[(transactions_train['t_year'].isin([2019])) &\
                   (transactions_train['t_month'].isin([9, 10])) ]

In [107]:
filtered_df['customer_years'] = filtered_df.groupby(['customer_id'])['t_year'].transform("nunique")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [108]:
analysis_df = pd.merge(filtered_df[filtered_df['customer_years'] > 0][['customer_id', 'article_id', 't_year', 't_month']],
                       articles[['article_id','prod_name', 'product_type_name', 
                                 'product_group_name', 
                                 'department_name', 
                                 'index_name', 
                                 'index_group_name',
                                 'section_name',
                                 'garment_group_name',
                                 'detail_desc'
                                ]],
                       left_on=['article_id'],
                       right_on=['article_id'],
                       how = 'inner'
                      )

In [109]:
print(len(analysis_df['product_type_name'].unique()) , len(analysis_df['department_name'].unique()))

114 234


In [110]:
analysis_df[analysis_df['product_type_name']=='Cardigan'][['product_type_name','department_name']].drop_duplicates().sort_values('product_type_name')

,product_type_name,department_name
13118,Cardigan,Knitwear
2114210,Cardigan,Knitwear inactive from s1
1967875,Cardigan,Girls Local Relevance
1956005,Cardigan,Jersey inactive from s1
1894555,Cardigan,Equatorial Assortment
1881418,Cardigan,Baby Boy Jersey Fancy
1801399,Cardigan,Outdoor/Blazers DS
1757327,Cardigan,Girls Projects
1716564,Cardigan,Baby basics
1704296,Cardigan,Kids Boy Jersey Fancy


In [111]:
df = analysis_df.groupby(['product_type_name','department_name']).agg({'customer_id':'nunique','t_year':'nunique'}).reset_index()

In [112]:
df.sort_values(['t_year', 'customer_id'], ascending=[False, False]).head(20)

,product_type_name,department_name,customer_id,t_year
971,Sweater,Knitwear,107927,1
1221,Trousers,Trouser,73608,1
72,Blouse,Blouse,49406,1
991,Sweater,Tops Knitwear,40844,1
1223,Trousers,Trousers,39409,1
243,Dress,Dress,37801,1
1173,Trousers,Denim Trousers,37140,1
140,Bra,Expressive Lingerie,27635,1
1113,Top,Jersey fancy,26921,1
495,Jacket,Outwear,23566,1


In [113]:
random_sample = analysis_df[['customer_id']].drop_duplicates().sample(frac=0.5, random_state=42)

In [114]:
analysis_df[(analysis_df['customer_id'].isin(random_sample['customer_id'].unique().tolist()))].\
groupby(['product_group_name','department_name','article_id']).\
agg(cust_count=('customer_id','nunique')).reset_index().sort_values('cust_count', ascending=False).head(10)

,product_group_name,department_name,article_id,cust_count
11922,Garment Lower body,Trousers,706016001,2497
19968,Garment Upper body,Knitwear,673677002,2227
13954,Garment Upper body,Basic 1,772902001,1841
11923,Garment Lower body,Trousers,706016002,1585
19971,Garment Upper body,Knitwear,673677010,1492
11924,Garment Lower body,Trousers,706016003,1389
10705,Garment Lower body,Trouser,562245046,1386
9244,Garment Lower body,Ladies Sport Bottoms,730683001,1283
9224,Garment Lower body,Ladies Sport Bottoms,720125001,1283
19873,Garment Upper body,Knitwear,660599017,1257


In [115]:
analysis_df[~(analysis_df['customer_id'].isin(random_sample['customer_id'].unique().tolist()))].\
groupby(['product_group_name','department_name','article_id']).\
agg(cust_count=('customer_id','count')).reset_index().sort_values('cust_count', ascending=False).head(10)

,product_group_name,department_name,article_id,cust_count
11879,Garment Lower body,Trousers,706016001,3343
19967,Garment Upper body,Knitwear,673677002,2406
13905,Garment Upper body,Basic 1,772902001,2153
11880,Garment Lower body,Trousers,706016002,2060
10644,Garment Lower body,Trouser,562245046,1708
11881,Garment Lower body,Trousers,706016003,1685
7643,Garment Lower body,Denim Trousers,714790003,1601
19970,Garment Upper body,Knitwear,673677010,1589
23298,Garment Upper body,Suit,568601006,1583
11551,Garment Lower body,Trouser,796210008,1579


In [156]:
top_12_products = transactions_train[(transactions_train['t_year']==2020)&
                                     (transactions_train['t_month'].isin([8,9]))
                                    ].groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(12)

In [157]:
pd.merge(top_12_products,
         articles[['article_id','prod_name', 'product_type_name', 
                                 'product_group_name', 
                                 'department_name', 
                                 'index_name', 
                                 'index_group_name',
                                 'section_name',
                                 'garment_group_name',
                                 'detail_desc'
                                ]],
                       left_on=['article_id'],
                       right_on=['article_id'],
                       how = 'inner'
        ).head()

,article_id,customer_id,prod_name,product_type_name,product_group_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc
0,751471001,3193,Pluto RW slacks (1),Trousers,Garment Lower body,Trouser,Ladieswear,Ladieswear,Womens Everyday Collection,Trousers,Ankle-length cigarette trousers in a stretch w...
1,918292001,3176,STRONG HW seamless tights,Leggings/Tights,Garment Lower body,Ladies Sport Bottoms,Sport,Sport,Ladies H&M Sport,Jersey Fancy,Sports tights in fast-drying functional fabric...
2,706016001,2763,Jade HW Skinny Denim TRS,Trousers,Garment Lower body,Trousers,Divided,Divided,Divided Collection,Trousers,High-waisted jeans in washed superstretch deni...
3,915526001,2555,Nika vest,Sweater,Garment Upper body,Knitwear,Ladieswear,Ladieswear,Womens Everyday Collection,Knitwear,"Relaxed-fit slipover in a soft, rib-knit cotto..."
4,916468003,2484,Bailey,Cardigan,Garment Upper body,Knitwear,Ladieswear,Ladieswear,Womens Everyday Collection,Knitwear,"Boxy, V-neck cardigan in a soft rib knit made ..."


In [158]:
top_12_products['article_id'] = " 0"+top_12_products.article_id.astype('str')

In [159]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [160]:
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [161]:
submission['prediction'] = top_12_products.article_id.sum()
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0751471001 0918292001 0706016001 0915526001 0...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0751471001 0918292001 0706016001 0915526001 0...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0751471001 0918292001 0706016001 0915526001 0...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0918292001 0706016001 0915526001 0...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0751471001 0918292001 0706016001 0915526001 0...


In [162]:
submission.to_csv('submission.csv', index = False)